In [56]:
#THaW Project
#This program is meant to establish the accuracy of predicting device name from the data collected by MSU
#6/12/2019
#Code Written By: Manzi Bryan with a lot of help from https://www.kaggle.com/nageshnaik/iris-dataset-classfication-using-naive-bayes
# This code should be submitted for review 

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
# from sklearn.neural_network import MLPClassifier

from sklearn.metrics import *
import sklearn
from sklearn import model_selection
from sklearn.model_selection import *
import pylab as pl
import pandas as pd
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import warnings
import pickle
import operator
import time
import seaborn as sns
from pprint import pprint# Look at parameters used by our current forest

target = 'Model'


# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets
# The dontTrainOn parameter is meant to tell the classifier to leave one tenth of one device out of training

# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets

def makeClassifier(df, functionName, clf, labels, f1Dataframe = None, i= None,):
    
    inTestingMode = not (f1Dataframe == None and i == None)
    devices = df[target].unique()
    copydf = df.copy()
    
    copydf[target].replace(devices, range(0, len(devices)), inplace=True)
        
    Y = copydf[target].tolist()
    
#     print("There are " + str(len(devices)) + " devices")
    
    #Remove labelling columns from the index
    copydf = copydf.drop(columns= labels)
    X = copydf.values
    
    
    #One tenth of the data as test
    validation_size = 0.1
    
    seed = 7
    
    X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, 
                                                    test_size=validation_size, random_state=seed)
    
    scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_macro': 'recall_macro',
            'f1_macro' : 'f1_macro'}
#     scoring = {
#             'acc': 'accuracy',
#             'f1_macro' : 'f1_macro'}

    
    #Fitting the training set
    clf.fit(X_train, Y_train) 
    
   
    #Model Performance
    #setting performance parameters
    kfold = model_selection.StratifiedKFold(n_splits=10, random_state=seed) #same number of samples from each 

    #calling the cross validation function
    
    cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring, return_train_score=True)
    

    
    if not inTestingMode:
        filename = functionName  + 'Model.sav' # Save Classifier
        print(functionName)
        pickle.dump(clf, open(filename, 'wb'))
        for metric in cv_results.keys():
            print(metric + ": " + str(cv_results[metric].mean()))
        print('\n')
        
        
    else:
        f1Dataframe['Classifier'][i] = functionName
        f1Dataframe['f1'][i] = str(cv_results['test_f1_macro'].mean())
    
    return clf
    


In [57]:
def makeOneClassifier(df, labels):
    n_est = 15
    depth = 15
    name = 'RandomForest'
    name2 = 'KNN'
    name3 = 'DecisionTree'
    name4 = 'GNB'
    
    classifier = RandomForestClassifier(bootstrap= True, max_depth= 30, max_features= 3, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 30, random_state=42)
    classifier3 = DecisionTreeClassifier(max_depth= 21, max_features= 3, min_samples_leaf= 3, min_samples_split= 3, random_state=42)
    classifier2 = KNeighborsClassifier(algorithm= 'auto', leaf_size= 1, n_neighbors= 11, weights= 'distance')
    classifier4 = GaussianNB()
    

    devices = df[target].unique()
#     df = pd.read_excel(path)

    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the dataframe
    makeClassifier(df, name, classifier, labels)
    makeClassifier(df, name2, classifier2, labels)
    makeClassifier(df, name3, classifier3, labels)
    makeClassifier(df, name4, classifier4, labels)
    return devices

In [58]:
def makeManyClassifiers(df, labels):
    
    functions = {}
    f1Dataframe = pd.DataFrame(index = range(0, 10000), columns=['Classifier', 'f1'])
    # Grid search from 1 to 100
    for depthDivideBy5 in range(1, 21):
        for n_estDivideBy5 in range(1, 21):
            depth = depthDivideBy5 * 5
            n_est = n_estDivideBy5 * 5
            functions['RandomForest d=' + str(depth) + ' n_est=' + str(n_est)] = RandomForestClassifier(n_estimators=n_est, max_depth=depth)
    
#     functions['RandomForest d=14 n_est=25'] = RandomForestClassifier(n_estimators=14, max_depth=25)
#     functions['RandomForest d=22 n_est=55'] = RandomForestClassifier(n_estimators=22, max_depth=55)
    devices = df[target].unique()
    i = 0
    for functionName in functions:
        df = pd.read_excel(path)
        
        df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the index
#         df.drop(df.columns[df.columns.str.contains('Bursts',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from 
        makeClassifier(df, functionName, functions[functionName], labels, f1Dataframe, i)
        i += 1
    f1Dataframe.to_excel('./BestClassifiers.xlsx')
    return devices

In [59]:
if __name__ == "__main__":
    
    path= 'efficientTrain.xlsx'
    
    print("About to read...")
    traindf = pd.read_excel(path) 
    print("...read")
    traindf = traindf.reindex(sorted(traindf.columns), axis=1)
    
    labels = ['Device','Model','App','Distance', 'Bursts']
    
#     duplicated = ['iHealth Blood Pressure', 'Fever Sense', 'iHealth gluco', 'Portable_ECG', 'Pyle Health']
    
#     df_filtered0 = traindf[traindf['Model'] == duplicated[0]] 
#     df_filtered1 = traindf[traindf['Model'] == duplicated[1]] 
#     df_filtered2 = traindf[traindf['Model'] == duplicated[2]] 
#     df_filtered3 = traindf[traindf['Model'] == duplicated[3]] 
#     df_filtered4 = traindf[traindf['Model'] == duplicated[4]] 
#     frames = [df_filtered0, df_filtered1, df_filtered2, df_filtered3, df_filtered4]
    
#     result = pd.concat(frames)
    
#     makeManyClassifiers(, labels) 
    makeOneClassifier(traindf, labels)
#     print("Done")


About to read...
...read
RandomForest
fit_time: 0.21381311416625975
score_time: 0.02028179168701172
test_acc: 0.9622260356424415
train_acc: 0.9997077588658237
test_prec_macro: 0.9534514935656728
train_prec_macro: 0.9996202613145421
test_rec_macro: 0.9378556184613899
train_rec_macro: 0.9994147517576847
test_f1_macro: 0.9439252044846024
train_f1_macro: 0.9995158616991724


KNN
fit_time: 0.012168788909912109
score_time: 0.10919499397277832
test_acc: 0.917951191701933
train_acc: 1.0
test_prec_macro: 0.9084968704712738
train_prec_macro: 1.0
test_rec_macro: 0.8770359203623975
train_rec_macro: 1.0
test_f1_macro: 0.885162085503354
train_f1_macro: 1.0


DecisionTree
fit_time: 0.011923789978027344
score_time: 0.003454756736755371
test_acc: 0.9222156287481991
train_acc: 0.9685697302674301
test_prec_macro: 0.8861169520956841
train_prec_macro: 0.9556353088044978
test_rec_macro: 0.8790130948046262
train_rec_macro: 0.9501151198958393
test_f1_macro: 0.8778326710622529
train_f1_macro: 0.952492710881456

C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 

GNB
fit_time: 0.00397794246673584
score_time: 0.007487988471984864
test_acc: 0.5942613596084588
train_acc: 0.5948317851614792
test_prec_macro: 0.499514824133116
train_prec_macro: 0.5058744329138889
test_rec_macro: 0.5350264994183649
train_rec_macro: 0.5341727060316128
test_f1_macro: 0.4521340282364469
train_f1_macro: 0.45265752589058944




C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\brnma\Miniconda3\envs\machinelearning\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
def makeHisto(confidences, deviceName):
    # Make Histograms
    bins = np.linspace(confidences[0], confidences[-1])
    
    num_bins = 100 # <-- Change here - Specify total number of bins for histogram
    plt.hist(confidences, bins=np.linspace(np.min(confidences), np.max(confidences), num=num_bins)) #<-- Change here.  Note the use of ravel.
    plt.savefig(deviceName + ' Histogram')
    plt.cla()

In [ ]:
def getNumerical(devices, y):
    i = 0
    
    for device in devices:
        if device == y[0]:
            return [i] * len(y)
        i += 1

def makePrediction(model, unseen, devices, numReadings, deviceName, labels):
    loadedModel = pickle.load(open(model, 'rb'))
    unseen = unseen.reindex(sorted(unseen.columns), axis=1)
    
    unseen.drop(unseen.columns[unseen.columns.str.contains('Unname',case = False)],axis = 1, inplace = True)
    
    y = unseen[target].tolist() #unseen should only contain one device, hence y is a list of the same device repeated
    
    
    unseen = unseen.drop(columns= labels)
    predictions = loadedModel.predict(unseen)
    
    predictions_proba = loadedModel.predict_proba(unseen)
    i = 0
    other = 0
    confidences = []
    for i in range(0, len(predictions_proba)):
        confidences.append(round(predictions_proba[i].max(), 3))
        
    print(len(predictions_proba))
    confidences.sort()